In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import re
from scipy.signal import butter, filtfilt

##### Gathering data

In [ ]:
# Control data
mmmp_dir = './Control'
fma_control = glob(os.path.join(mmmp_dir, '**/Turning_FMA0[0-9]_C[0-9]*.tsv'), recursive=True)


In [ ]:
#PwP data
mmmp_dir = './PWP'
fma_pwp = glob(os.path.join(mmmp_dir, '**/Turning_FMA0[0-9]_P[0-9]*.tsv'), recursive=True)

In [ ]:
# Data passes thorugh low-pass butterworth filter of 4th order
def butter_lowpass_filter(data, cutoff_frequency=10, sampling_rate=100, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data, axis=0, padlen=0)
    return filtered_data

##### Reading file function

In [ ]:
NAN_special = -999999
def read_turn_file(file):
    # Load the motion capture data into a Pandas DataFrame
    df = pd.read_csv(file,sep = '\t',skiprows=11)
    df = df.fillna(NAN_special)
    df.columns = df.columns.str.strip() #eliminating unecesary spaces in column titles
    time =  df.loc[:, "Time"].to_numpy()
    
   
    marker_list = ['RSHO X', 'RSHO Y', 'RSHO Z', 'LSHO X', 'LSHO Y','LSHO Z', 
                   'LSTC X', 'LSTC Y', 'LSTC Z', 'RBAK X', 'RBAK Y','RBAK Z',
                   'LFHD X', 'LFHD Y', 'LFHD Z', 'RFHD X', 'RFHD Y', 'RFHD Z',
                   'LBHD X', 'LBHD Y', 'LBHD Z', 'RBHD X', 'RBHD Y', 'RBHD Z',
                   'STRN X', 'STRN Y', 'STRN Z',
                   'LASI X', 'LASI Y', 'LASI Z', 'RASI X', 'RASI Y', 'RASI Z',
                   'LPSI X', 'LPSI Y', 'LPSI Z', 'RPSI X', 'RPSI Y', 'RPSI Z',
                  'LTOE X', 'LTOE Y', 'LTOE Z','RTOE X', 'RTOE Y', 'RTOE Z',
                  'LANK X', 'LANK Y', 'LANK Z','RANK X', 'RANK Y', 'RANK Z',
                  'LHEE X', 'LHEE Y', 'LHEE Z','RHEE X', 'RHEE Y', 'RHEE Z']

    # create empty arrays for each marker
    RSHO = np.zeros((len(time), 3))
    LSHO = np.zeros((len(time), 3))
    LSTC = np.zeros((len(time), 3))
    RBAK = np.zeros((len(time), 3))
    LFHD = np.zeros((len(time), 3))
    RFHD = np.zeros((len(time), 3))
    LBHD = np.zeros((len(time), 3))
    RBHD = np.zeros((len(time), 3))
    STRN = np.zeros((len(time), 3))
    LASI = np.zeros((len(time), 3))
    RASI = np.zeros((len(time), 3))
    LPSI = np.zeros((len(time), 3))
    RPSI = np.zeros((len(time), 3))
    RTOE = np.zeros((len(time), 3))
    LTOE = np.zeros((len(time), 3))
    LANK = np.zeros((len(time), 3))
    RANK = np.zeros((len(time), 3))
    LHEE = np.zeros((len(time), 3))
    RHEE = np.zeros((len(time), 3))
    
    flag = False # this flAg will help us identify the files that are incomplete later 
    missing_markers = []
    
    for col_name in marker_list:
        if col_name in df.columns:
            
            if col_name.startswith('RSHO'):
                RSHO = df.loc[:, ['RSHO X', 'RSHO Y', 'RSHO Z']].to_numpy()
                RSHO = butter_lowpass_filter(RSHO)
            elif col_name.startswith('LSHO'):
                LSHO = df.loc[:, ['LSHO X', 'LSHO Y', 'LSHO Z']].to_numpy()
                LSHO = butter_lowpass_filter(LSHO)
            elif col_name.startswith('LSTC'):
                LSTC = df.loc[:, ['LSTC X', 'LSTC Y', 'LSTC Z']].to_numpy()
                LSTC = butter_lowpass_filter(LSTC)
            elif col_name.startswith('RBAK'):
                RBAK = df.loc[:, ['RBAK X', 'RBAK Y', 'RBAK Z']].to_numpy()
                RBAK = butter_lowpass_filter(RBAK)
            elif col_name.startswith('LFHD'):
                LFHD = df.loc[:, ['LFHD X', 'LFHD Y', 'LFHD Z']].to_numpy()
                LFHD = butter_lowpass_filter(LFHD)
            elif col_name.startswith('RFHD'):
                RFHD = df.loc[:, ['RFHD X', 'RFHD Y', 'RFHD Z']].to_numpy()
                RFHD = butter_lowpass_filter(RFHD)
            elif col_name.startswith('LBHD'):
                LBHD = df.loc[:, ['LBHD X', 'LBHD Y', 'LBHD Z']].to_numpy()
                LBHD = butter_lowpass_filter(LBHD)
            elif col_name.startswith('RBHD'):
                RBHD = df.loc[:, ['RBHD X', 'RBHD Y', 'RBHD Z']].to_numpy()
                RBHD = butter_lowpass_filter(RBHD)
            elif col_name.startswith('STRN'):
                STRN = df.loc[:, ['STRN X', 'STRN Y', 'STRN Z']].to_numpy()
                STRN = butter_lowpass_filter(STRN)
            elif col_name.startswith('LASI'):
                LASI = df.loc[:, ['LASI X', 'LASI Y', 'LASI Z']].to_numpy()
                LASI = butter_lowpass_filter(LASI)
            elif col_name.startswith('RASI'):
                RASI = df.loc[:, ['RASI X', 'RASI Y', 'RASI Z']].to_numpy()
                RASI = butter_lowpass_filter(RASI)
            elif col_name.startswith('LPSI'):
                LPSI = df.loc[:, ['LPSI X', 'LPSI Y', 'LPSI Z']].to_numpy()
                LPSI = butter_lowpass_filter(LPSI)
            elif col_name.startswith('RPSI'):
                RPSI = df.loc[:, ['RPSI X', 'RPSI Y', 'RPSI Z']].to_numpy()
                RPSI = butter_lowpass_filter(RPSI)
            elif col_name.startswith('RTOE'):
                RTOE = df.loc[:, ['RTOE X', 'RTOE Y', 'RTOE Z']].to_numpy()
                RTOE = butter_lowpass_filter(RTOE)
            elif col_name.startswith('LTOE'):
                LTOE = df.loc[:, ['LTOE X', 'LTOE Y', 'LTOE Z']].to_numpy()
                LTOE = butter_lowpass_filter(LTOE)
            elif col_name.startswith('LANK'):
                LANK = df.loc[:, ['LANK X', 'LANK Y', 'LANK Z']].to_numpy()
                LANK = butter_lowpass_filter(LANK)
            elif col_name.startswith('RANK'):
                RANK = df.loc[:, ['RANK X', 'RANK Y', 'RANK Z']].to_numpy()
                RANK = butter_lowpass_filter(RANK)
            elif col_name.startswith('LHEE'):
                LHEE = df.loc[:, ['LHEE X', 'LHEE Y', 'LHEE Z']].to_numpy()
                LHEE = butter_lowpass_filter(LHEE)
            elif col_name.startswith('RHEE'):
                RHEE = df.loc[:, ['RHEE X', 'RHEE Y', 'RHEE Z']].to_numpy()
                RHEE = butter_lowpass_filter(RHEE)
        else:
            flag = True
            missing_markers.append(col_name)
    
#     print('File: {} , does not have markers {}'.format(file, missing_markers))
            
    return flag,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE

## Yaw angles per segment (Head, Shoulders,Pelvis, Feet)

In [ ]:

flag,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE = read_turn_file(file)

def segment_yaw_angle(marker_init,marker_final):
    """
    Calculate the yaw angle of vectors in the xy-plane relative to the y-axis.

    Parameters:
    marker_init (numpy.ndarray): 2D vectors (N_samples x 2).

    Returns:
    numpy.ndarray: Yaw angles in degrees (N_samples).
    
    """
    # Get 2D (xy) coordinates of the vectors for their projection in the xy plane
    marker_init_xy = marker_init[:, :2]
    marker_final_xy = marker_final[:, :2]
    # Define segment vector
    segment = marker_final_xy-marker_init_xy
    
    # Normalize the vectors
    segment_norm = segment / np.linalg.norm(segment, axis=1, keepdims=True)

    # Unit vector in the y-axis 
    y_axis_unit = np.array([0, 1])

    dot_product = np.einsum('ij,j->i', segment_norm, y_axis_unit)  

    # Calculate the angle in radians and then convert to degrees
    yaw_angle_radians = np.arccos(np.clip(dot_product, -1.0, 1.0))
    
    # Obtain yaw sign with cross product
    cross_product = np.cross(segment_norm, y_axis_unit)
    sign = np.sign(cross_product)
    
    yaw_angle_degrees = np.degrees(yaw_angle_radians) * sign
        
    return  yaw_angle_degrees


def yaw_angle_per_segment(left_forehead, right_forehead,left_shoulder, right_shoulder,left_hip, right_hip,left_ankle,right_ankle):
    """
    Calculate the yaw angle for all segment

    Returns:
    numpy.ndarray x 3 : Yaw angles of all segments.
    
    """
    # we define the turning convention from left to right, so right marker = final, left marker = init
    head_yaw = segment_yaw_angle(left_forehead,right_forehead)
    shoulders_yaw = segment_yaw_angle(left_shoulder,right_shoulder)
    pelvis_yaw = segment_yaw_angle(left_hip,right_hip)
    feet_yaw = segment_yaw_angle(left_ankle,right_ankle)
    
    return head_yaw,shoulders_yaw,pelvis_yaw,feet_yaw

head_yaw,shoulders_yaw,pelvis_yaw,feet_yaw = yaw_angle_per_segment(LFHD, RFHD,LSHO, RSHO,LASI, RASI,LANK,RANK)


## Angular velocity and Angural Jerk

In [ ]:
def get_angular_velocity(segment_yaw_angle,time):
    time_diff = np.diff(time)
    
    # Angular velocity: first derivative of yaw angle
    angular_velocity = np.diff(segment_yaw_angle) / time_diff
    
    return np.abs(angular_velocity) # we dont care about the sign because we dont care about the sense of the turn


def get_angular_jerk(angular_velocity, time):
    time_diff = np.diff(time)
    # 
    angular_acceleration = np.diff(angular_velocity) / time_diff[:-1]
    
    # 
    mean_angular_jerk = np.mean(np.diff(angular_acceleration) / time_diff[:-2])
    
    return mean_angular_jerk


head_ang_vel = get_angular_velocity(head_yaw,time)
shoulders_ang_vel = get_angular_velocity(shoulders_yaw,time)
pelvis_ang_vel = get_angular_velocity(pelvis_yaw,time)


mean_pelvis_jerk = get_angular_jerk(pelvis_ang_vel,time)
# # Calculate RMS of acceleration
# pelvis_jerk_rms = np.sqrt(np.mean(pelvis_jerk**2))

In [ ]:
plt.plot(time[:-1],head_ang_vel,label='head')
plt.plot(time[:-1],shoulders_ang_vel,label='shoulders')
plt.plot(time[:-1],pelvis_ang_vel,label='pelvis')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Angular velocity deg/s')

## Absolute amplitude of yaw angle rotation for each segment

In [ ]:
def calculate_yaw_amplitude(head_yaw, shoulders_yaw, pelvis_yaw):
    """
    Calculate the absolute amplitude of yaw rotation for each segment.

    Parameters:
    - head_yaw: numpy array of yaw angles for the head
    - shoulders_yaw: numpy array of yaw angles for the shoulders
    - pelvis_yaw: numpy array of yaw angles for the pelvis
    - feet_yaw: numpy array of yaw angles for the feet

    Returns:
    - A dictionary containing the amplitude for each segment.
    """
    head_amplitude = np.max(head_yaw) - np.min(head_yaw)
    shoulders_amplitude = np.max(shoulders_yaw) - np.min(shoulders_yaw)
    pelvis_amplitude = np.max(pelvis_yaw) - np.min(pelvis_yaw)
    

    return {
        'head_amplitude': head_amplitude,
        'shoulders_amplitude': shoulders_amplitude,
        'pelvis_amplitude': pelvis_amplitude   
    }

absolute_amplitudes = calculate_yaw_amplitude(head_yaw, shoulders_yaw, pelvis_yaw)
print("Yaw Rotation Amplitudes:")
print(f"Head: {absolute_amplitudes['head_amplitude']:.2f}°")
print(f"Shoulders: {absolute_amplitudes['shoulders_amplitude']:.2f}°")
print(f"Pelvis: {absolute_amplitudes['pelvis_amplitude']:.2f}°")




In [ ]:
def calculate_relative_yaw_angles_from_amplitudes(absolute_amplitudes):
    """
    Calculate the relative yaw rotation angles between adjacent segments using the absolute yaw amplitudes:
    - Head relative to shoulders
    - Shoulders relative to pelvis
    - Head relative to pelvis
    
    Parameters:
    - absolute_amplitudes: A dictionary containing the absolute yaw amplitudes for the head, shoulders, and pelvis.
    
    Returns:
    - A dictionary containing the relative yaw angles for each pair of segments.
    """
    # Extract absolute amplitudes from the input dictionary
    head_amplitude = absolute_amplitudes['head_amplitude']
    shoulders_amplitude = absolute_amplitudes['shoulders_amplitude']
    pelvis_amplitude = absolute_amplitudes['pelvis_amplitude']
    
    # Calculate the relative yaw rotation: head - shoulders, shoulders - pelvis, head - pelvis
    head_relative_shoulders = head_amplitude - shoulders_amplitude
    shoulders_relative_pelvis = shoulders_amplitude - pelvis_amplitude
    head_relative_pelvis = head_amplitude - pelvis_amplitude
    
    return {
        'head_relative_shoulders': head_relative_shoulders,
        'shoulders_relative_pelvis': shoulders_relative_pelvis,
        'head_relative_pelvis': head_relative_pelvis
    }


relative_angles = calculate_relative_yaw_angles_from_amplitudes(absolute_amplitudes)


print("Relative Yaw Angles (based on absolute amplitudes):")

print(f"Head relative to Shoulders: {relative_angles['head_relative_shoulders']:.2f}")
print(f"Shoulders relative to Pelvis: {relative_angles['shoulders_relative_pelvis']:.2f}")
print(f"Head relative to Pelvis: {relative_angles['head_relative_pelvis']:.2f}")


## First stride detection

In [ ]:
def calculate_velocity(marker_pos, time):
    """Calculate velocity based on position data."""
    delta_pos = np.diff(marker_pos, axis=0)
    delta_time = np.diff(time)
    velocity = delta_pos[:, 1] / delta_time  # Calculate forward (y-axis) velocity
    return velocity


def detect_first_stride(LTOE, RTOE, time):
    """
    Detect the first stride onset and end based on forward velocity of toe markers.
    
    Parameters:
    LTOE, RTOE: numpy arrays with shape (N_samples, 3) representing the 3D positions of the left and right toes.
    time: numpy array of time points.

    Returns:
    - stride_label: Which foot initiated the stride ('left' or 'right').
    - onset_index, onset_time: Index and time of the first detected stride onset.
    - end_index, end_time: Index and time of the end of the stride.
    """
    # Calculate the forward (x-axis) velocities for both feet
    LTOE_velocity = calculate_velocity(LTOE, time)
    RTOE_velocity = calculate_velocity(RTOE, time)

    # Find the peak index for both feet
    LTOE_peak_index = np.argmax(LTOE_velocity)
    RTOE_peak_index = np.argmax(RTOE_velocity)
    
    # Define thresholds based on peak velocity minus 2 * standard deviation
    LTOE_threshold = LTOE_velocity[LTOE_peak_index] - 2 * np.std(LTOE_velocity)
    RTOE_threshold = RTOE_velocity[RTOE_peak_index] - 2 * np.std(RTOE_velocity)
    
    # Detect the stride onset (left tail) for both feet
    L_stride_index = LTOE_peak_index
    while L_stride_index > 0 and LTOE_velocity[L_stride_index] > LTOE_threshold:
        L_stride_index -= 1

    R_stride_index = RTOE_peak_index
    while R_stride_index > 0 and RTOE_velocity[R_stride_index] > RTOE_threshold:
        R_stride_index -= 1

    # Determine which foot initiated the stride
    if L_stride_index < R_stride_index:
        onset_index = L_stride_index
        velocity = LTOE_velocity
        stride_label = 'left'
    else:
        onset_index = R_stride_index
        velocity = RTOE_velocity
        stride_label = 'right'

    onset_time = time[onset_index]

    # Detect the stride end (right tail)
    # Use a small threshold to detect when velocity stabilizes close to zero
    stable_threshold = 50  # Velocity threshold (mm/s)
    end_index = onset_index + 1

    while end_index < len(velocity) - 1 and abs(velocity[end_index]) > stable_threshold:
        end_index += 1

    if end_index < len(time):
        end_time = time[end_index]
    else:
        end_time = time[-1]

    stride_duration = end_time - onset_time

    return stride_label, onset_index, onset_time, end_index, end_time, stride_duration


stride_label, onset_index, onset_time, end_index, end_time, stride_duration = detect_first_stride(LTOE, RTOE, time)

print(f"First stride detected with {stride_label} foot at index {onset_index} (time: {onset_time:.2f}s), "
      f"end at index {end_index} (time: {end_time:.2f}s), total stride duration: {stride_duration:.2f}s")

## Segment Rotation Onset time

The rotation onsets of each segment (head, trunk, pelvis) are expressed as percentages of the gait cycle, specifically normalized to the first stride (Hong et al., 2008)

In [ ]:
def detect_turn_onset_3std(angular_velocity, time, threshold_factor=3):
    """
    Detect the turn onset by finding the left tail of the Gaussian-like curve
    at 3 * standard deviation from the peak.

    Parameters:
    angular_velocity (numpy.ndarray): Angular velocity in degrees per second.
    time (numpy.ndarray): Time vector in seconds.
    threshold_factor (float): Factor to multiply by std to define the threshold for onset (default 3).

    Returns:
    int: Index of the turn onset.
    """
    # Find the peak index in the angular velocity curve
    peak_index = np.argmax(angular_velocity)
    
    # Set the threshold as peak value minus threshold_factor * std of the angular velocity
    threshold = angular_velocity[peak_index] - threshold_factor * np.std(angular_velocity)
    
    # Search for the left tail starting index
    left_tail_index = peak_index  # Start at the maximum point (peak)
    while left_tail_index > 0 and angular_velocity[left_tail_index] > threshold:
        left_tail_index -= 1  # Move one position backwards until the value is below the threshold
    
    # Return the time of the onset (the time corresponding to the left_tail_index)
    onset_time = time[left_tail_index]  # Use time at the onset index
    return left_tail_index, onset_time



head_yaw,shoulders_yaw,pelvis_yaw,feet_yaw = yaw_angle_per_segment(LFHD, RFHD,LSHO, RSHO,LASI, RASI,LANK,RANK)


head_ang_vel = get_angular_velocity(head_yaw,time)
shoulders_ang_vel = get_angular_velocity(shoulders_yaw,time)
pelvis_ang_vel = get_angular_velocity(pelvis_yaw,time)

# Detect onsets for each segment
head_index, head_onset = detect_turn_onset_3std(head_ang_vel, time)
shoulders_index, shoulders_onset = detect_turn_onset_3std(shoulders_ang_vel, time)
pelvis_index, pelvis_onset = detect_turn_onset_3std(pelvis_ang_vel, time)


print("yaw rotation onset times:")
print(f"Head: {head_onset:.2f} s")
print(f"Shoulders: {shoulders_onset:.2f} s")
print(f"Pelvis: {pelvis_onset:.2f} s")




## Segment onset percentage

$$
\text { Onset Percentage }=\left(\frac{\text { Yaw onset time }- \text { First stride onset time }}{\text { End first stride time }}\right) \times 100
$$

We normalize the segment onset times relative to the duration of the first stride. 

By using the first stride duration as the reference, we express the yaw onset times of the head, shoulders, and pelvis as percentages relative to the time the initial stride ended. 


In [ ]:

def calculate_onset_percentage(onset_time, first_stride_time, total_first_stride_duration):
    """
    Calculate the onset time as a percentage relative to the first stride onset.
    
    Parameters:
    - onset_time: The detected onset time of the yaw rotation (in seconds).
    - first_stride_time: The time of the first stride onset (in seconds).
    - total_trial_duration: The total duration of the trial (in seconds).
    
    Returns:
    - Onset time as a percentage of the first stride onset.
    """
    return ((onset_time - first_stride_time) / total_first_stride_duration) * 100


stride_label, onset_index, first_stride_time, end_index, end_stride_time, stride_durationn = detect_first_stride(LTOE, RTOE, time)

print(stride_duration)
# Calculate onset times as percentages relative to the first stride onset
head_onset_pct = calculate_onset_percentage(head_onset, first_stride_time, end_stride_time)
shoulders_onset_pct = calculate_onset_percentage(shoulders_onset, first_stride_time, end_stride_time)
pelvis_onset_pct = calculate_onset_percentage(pelvis_onset, first_stride_time, end_stride_time)

print("Normalized yaw rotation onset times (relative to the first stride onset):")
print(f"Head onset: {head_onset_pct:.2f}%")
print(f"Shoulders onset: {shoulders_onset_pct:.2f}%")
print(f"Pelvis onset: {pelvis_onset_pct:.2f}%")

## Peak sagittal plane inclination (yz plane)

     The peak sagittal inclination is calculated with the yz coordiantes of the sternum marker. The max inclination represents the max deviation of the sternum in the sagittal plane from its initial position at the start of the turn. 

In [ ]:
def calculate_peak_sagittal_inclination(sternum):

        # Calculate the inclination angle with respect to the yz plane for each vector
    inclination_angle = np.arctan2(sternum[:, 2], sternum[:, 1])

    # Adjust the angle to be positive in the yz plane for each vector. 90 degrees are removed from the calculation to obtain the
    # angle wrt to the z axis. 
    inclination_angle_degrees = np.where(inclination_angle < 0,
                                         90 - abs(np.degrees(inclination_angle)),
                                         90 - np.degrees(inclination_angle))

    inclination_angle_degrees -= inclination_angle_degrees[0]
    peak_inclination = max(inclination_angle_degrees)
    
    return peak_inclination #,inclination_angle_degrees


peak_inclination = calculate_peak_sagittal_inclination(STRN)
print(peak_inclination)

## Peak inclination in the frontal plane (xz plane)

    The peak frontal inclination is calculated with the xz coordiantes of the sternum marker. The max inclination represents the max deviation of the sternum in the frontal plane from its initial position at the start of the turn. 

In [ ]:
def calculate_peak_frontal_inclination(sternum):

        # Calculate the inclination angle with respect to the xz plane for each vector
    inclination_angle = np.arctan2(sternum[:, 2], sternum[:, 0])

    # Adjust the angle to be positive in the yz plane for each vector. 90 degrees are removed from the calculation to obtain the
    # angle wrt to the z axis. 
    inclination_angle_degrees = np.where(inclination_angle < 0,
                                         90 - abs(np.degrees(inclination_angle)),
                                         90 - np.degrees(inclination_angle))
#     inclination_angle_degrees = np.degrees(inclination_angle)
    inclination_angle_degrees -= inclination_angle_degrees[0]
    peak_frontal_inclination = max(np.abs(inclination_angle_degrees))
    
    return peak_frontal_inclination #,inclination_angle_degrees

peak_frontal_inclination = calculate_peak_frontal_inclination(STRN)

print(peak_frontal_inclination)

### Toe-Off and Heel-Strike

In [ ]:
def calculate_foot_angle(marker1, marker2):
    # Calculate the vector between Marker 1 and Marker 2
    vector = marker2 - marker1

    # Calculate the hypotenuse (c)
    c = np.linalg.norm(vector)

    # Calculate z component
    z = marker1[2] - marker2[2]

    # Calculate theta
    theta = np.degrees(np.arcsin(z / c))
    
    # Determine if it's toe-off angle (TO) or heel strike angle (HS)
    if theta > 0:
        result = "HS"  #  Heel strike angle
    else:
        result = "TO"  #  Toe-off angle
        theta = np.abs(theta)
        
    return theta, result

def calculate_leg_opposite(theta, marker1, marker2):
    # Calculate the leg opposite to the theta angle
    leg_opposite = np.tan(theta) * np.linalg.norm(marker2 - marker1)
    return leg_opposite

# Create a DataFrame with 'theta', 'result', and 'height' columns
theta_result_df = pd.DataFrame(columns=['theta', 'result', 'height', 'leg_opposite'])

for marker1, marker2 in zip(LTOE, LHEE):
    theta, result = calculate_foot_angle(marker1, marker2)
    leg_opposite = calculate_leg_opposite(theta, marker1, marker2)
    height_label = "Feet height" if result == "HS" else "Heel height"
    
    new_data_angle = {'theta': theta, 'result': result, 'height': height_label, 'leg_opposite': leg_opposite}
    
    # Add the new line to the DataFrame
    theta_result_df.loc[len(theta_result_df)] = new_data_angle

# Print the DataFrame with 'theta', 'result', and 'leg oppisite' columns
theta_result_df.head()


## Storing data in data frame

In [ ]:
#Create the dataframes to store the shoulder tilt and stroop angles
df_turning = pd.DataFrame(columns=['Subject ID','Group','TrialFMA',
                                   'Turn_duration[s]',
                                    'Mean_pelvis_jerk[mm/s3]','Absolute_head_yaw[deg]','Absolute_trunk_yaw[deg]','Absolute_pelvis_yaw[deg]',
                                    'Head_relative_to_trunk[deg]','Trunk_relative_to_pelvis[deg]','Head_relative_to_pelvis[deg]',
                                    'Head_Onset[%]', 'Trunk_Onset[%]', 'Pelvis_Onset[%]', 
                                     'Peak_Saggital_inclination[deg]','Peak_frontal_inclination[deg]',
                                    'Mean_TO[deg]','Mean_HS[deg]','Mean_Foot_height[mm]'])

full_data = [fma_control,fma_pwp]
data_label = ['Control','PwP']

for i,group in enumerate(full_data):
    type_subject = data_label[i]
    for file in group:
                
       # Extract subject ID
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.tsv$', file).group(1)
            subject_id = 'C'+subject_id
            color = 'blue'
        else: 
            subject_id = re.search(r'_P(\d+)\.tsv$', file).group(1)
            subject_id = 'P'+subject_id


        # Extract trial ID
        trial = re.search(r'_FMA0(\d+)_', file).group(1)

        # Load the motion capture data using the read_file function
        flag,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE = read_turn_file(file)

        # ------------------------- TURN DURATION
        
        total_trial_duration = time[-1] - time[0]
        
        #-------------------------
        
        
        # ------------------------- CALCULATE YAW ANGLES FOR EACH SEGMENT
        
        head_yaw,shoulders_yaw,pelvis_yaw,feet_yaw = yaw_angle_per_segment(LFHD, RFHD,LSHO, RSHO,LASI, RASI,LANK,RANK)
        
        #-------------------------
        
        # ------------------------- PELVIS JERK
        pelvis_ang_vel = get_angular_velocity(pelvis_yaw,time)
        
        mean_pelvis_jerk = get_angular_jerk(pelvis_ang_vel,time)
        #-------------------------
        
        
        # ------------------------- ABSOLUTE AMPLITUDE OF YAW ANGLES FOR EACH SEGMENT
        
        absolute_amplitudes = calculate_yaw_amplitude(head_yaw, shoulders_yaw, pelvis_yaw)

        #-------------------------
        
        
        # ------------------------- YAW ROTATION ANGLES BETWEEN SEGMENTS
        
        relative_angles = calculate_relative_yaw_angles_from_amplitudes(absolute_amplitudes)
        #-------------------------
        
        
        
        #------------------------- ONSET PERCENTAGE PER SEGMENT WRT FIRST STRIDE ONSET TIME
        
        # Detect onsets for each segment
        head_index, head_onset = detect_turn_onset_3std(head_ang_vel, time)
        shoulders_index, shoulders_onset = detect_turn_onset_3std(shoulders_ang_vel, time)
        pelvis_index, pelvis_onset = detect_turn_onset_3std(pelvis_ang_vel, time)

        stride_label, onset_index, first_stride_time, end_index, end_stride_time, stride_durationn = detect_first_stride(LTOE, RTOE, time)


        # Calculate onset times as percentages relative to the first stride onset
        head_onset_pct = calculate_onset_percentage(head_onset, first_stride_time, end_stride_time)
        shoulders_onset_pct = calculate_onset_percentage(shoulders_onset, first_stride_time, end_stride_time)
        pelvis_onset_pct = calculate_onset_percentage(pelvis_onset, first_stride_time, end_stride_time)

       
        #-------------------------
        
        #------------------------- PEAK SAGITTAL INCLINATION ANGLE 
        
        peak_sagittal = calculate_peak_sagittal_inclination(STRN)
        
        #-------------------------
        
        #------------------------- PEAK FRONTAL INCLINATION ANGLE 
        
        peak_frontal = calculate_peak_frontal_inclination(STRN)
        
        #-------------------------
        
        
        #------------------------- TO AND HS ANGLES
        
        left_theta_result_df = pd.DataFrame(columns=['theta', 'result', 'height', 'leg_opposite'])
        for marker1, marker2 in zip(LTOE, LHEE):
            theta, result = calculate_foot_angle(marker1, marker2)
            leg_opposite = calculate_leg_opposite(theta, marker1, marker2)
            height_label = "Feet height" if result == "HS" else "Heel height"

            new_data_langle = {'theta': theta, 'result': result, 'height': height_label, 'leg_opposite': leg_opposite}

            # Add the new line to the DataFrame
            left_theta_result_df.loc[len(left_theta_result_df)] = new_data_langle
        
        toe_off_df = left_theta_result_df[left_theta_result_df['result'] == 'TO']
        heel_strike_df = left_theta_result_df[left_theta_result_df['result'] == 'HS']
        
        #Left TO
        left_to = toe_off_df['theta'].mean(numeric_only=True)
        #Left HS
        left_hs = heel_strike_df['theta'].mean(numeric_only=True)
        #Left Foot height
        left_FH = heel_strike_df['leg_opposite'].mean(numeric_only=True)
        
        right_theta_result_df = pd.DataFrame(columns=['theta', 'result', 'height', 'leg_opposite'])
        for marker1, marker2 in zip(RTOE, RHEE):
            theta, result = calculate_foot_angle(marker1, marker2)
            leg_opposite = calculate_leg_opposite(theta, marker1, marker2)
            height_label = "Feet height" if result == "TO" else "Heel height"

            new_data_rangle = {'theta': theta, 'result': result, 'height': height_label, 'leg_opposite': leg_opposite}

            # Add the new line to the DataFrame
            right_theta_result_df.loc[len(right_theta_result_df)] = new_data_rangle
            
        r_toe_off_df = right_theta_result_df[right_theta_result_df['result'] == 'TO']
        r_heel_strike_df = right_theta_result_df[right_theta_result_df['result'] == 'HS']
        
        #Right TO
        right_to = r_toe_off_df['theta'].mean(numeric_only=True)
        #Right HS
        right_hs = r_heel_strike_df['theta'].mean(numeric_only=True)
        #Right Foot height
        right_FH = r_heel_strike_df['leg_opposite'].mean(numeric_only=True)
        
        #Mean TO,
        mean_TO = np.mean([left_to,right_to])
        #Mean HS
        mean_HS = np.mean([left_hs,right_hs])
        #Mean FH
        mean_FH = np.mean([left_FH,right_FH])
        

        #------------------------- SAVING DATA IN DF
        
        new_line = {'Subject ID':subject_id,'Group':type_subject,'TrialFMA':trial,
                     'Turn_duration[s]': total_trial_duration,
                    'Mean_pelvis_jerk[mm/s3]':mean_pelvis_jerk,
                    'Absolute_head_yaw[deg]':absolute_amplitudes['head_amplitude'],
                    'Absolute_trunk_yaw[deg]':absolute_amplitudes['shoulders_amplitude'],
                    'Absolute_pelvis_yaw[deg]':absolute_amplitudes['pelvis_amplitude'],
                    'Head_relative_to_trunk[deg]':relative_angles['head_relative_shoulders'],
                    'Trunk_relative_to_pelvis[deg]':relative_angles['shoulders_relative_pelvis'],
                    'Head_relative_to_pelvis[deg]':relative_angles['head_relative_pelvis'],
                    'Head_Onset[%]':head_onset_pct, 
                    'Trunk_Onset[%]':shoulders_onset_pct, 
                    'Pelvis_Onset[%]':pelvis_onset_pct, 
                    'Peak_Saggital_inclination[deg]':peak_sagittal,
                    'Peak_frontal_inclination[deg]':peak_frontal,
                    'Mean_TO[deg]':mean_TO,
                    'Mean_HS[deg]':mean_HS,
                    'Mean_Foot_height[mm]':mean_FH}

        df_turning.loc[len(df_turning)] = new_line
        
        

In [ ]:
pd.set_option('display.max_columns', None)
df_turning

In [ ]:
# df_turning.to_csv(r'C:\Users\maris\Documents\MMMP\Dataframes July 2024\turning_1024.csv')

In [ ]:
# df_turning_mean_fma.to_csv('./maris/Documents/MMMP/turning_mean_fmas.csv')

In [ ]:
# #Separate the files per FMA trial 1 for control subjects
# turning_fma1_c = [fp for fp in fma1_control if os.path.basename(fp).startswith("Turning")]
# turning_fma2_c = [fp for fp in fma2_control if os.path.basename(fp).startswith("Turning")]
# turning_fma3_c = [fp for fp in fma3_control if os.path.basename(fp).startswith("Turning")]

In [ ]:
# #Separate the files per FMA trial for pwp
# turning_fma1_p = [fp for fp in fma1_pwp if os.path.basename(fp).startswith("Turning")]
# turning_fma2_p = [fp for fp in fma2_pwp if os.path.basename(fp).startswith("Turning")]
# turning_fma3_p = [fp for fp in fma3_pwp if os.path.basename(fp).startswith("Turning")]

## Display segment turning on xy plane

In [ ]:
# def yaw_to_xy(yaw_angles, r):
#     """
#     Parameters:
#     yaw_angles: yaw angle of the segment
#     r: radius for the plot, so when plot together the yaw trajectories don't overlap
    
#     Returns: 
#     Plot of the segment trajectory over the transveral (xy) plane
#     """
#     yaw_angles_rad = np.radians(yaw_angles)  
#     x_coords = r * np.cos(yaw_angles_rad)
# #     x_coords = np.cos(yaw_angles_rad)
#     y_coords = r * np.sin(yaw_angles_rad)
# #     y_coords = np.sin(yaw_angles_rad)
#     return x_coords, y_coords

# # Convert yaw angles to XY coordinates
# plt.figure(figsize=(5, 5))
# head_x, head_y = yaw_to_xy(head_yaw,2.5)
# shoulders_x, shoulders_y = yaw_to_xy(shoulders_yaw,2)
# pelvis_x, pelvis_y = yaw_to_xy(pelvis_yaw,1.5)
# feet_x,feet_y = yaw_to_xy(feet_yaw,1)


# # Plot the trajectory in the XY plane
# # plt.figure(figsize=(10, 10))
# plt.plot(head_x, head_y, label='Head Yaw Trajectory', color='blue')
# plt.plot(shoulders_x, shoulders_y, label='Shoulders Yaw Trajectory', color='green')
# plt.plot(pelvis_x, pelvis_y, label='Pelvis Yaw Trajectory', color='red')
# plt.plot(feet_x,feet_y,label='Feet Yaw Trajectory',color='black')
# plt.scatter([0], [0], color='black', marker='x', label='Origin (0,0)')
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.legend()

### Onset determination strategy Version 1 (no longer used)

In [ ]:
# def detect_turn_onset(angular_velocity, time, threshold=0.05):
#     """
#     Detect the turn onset by finding where the slope of angular velocity changes
#     significantly at the left tail of the Gaussian-like curve.

#     Parameters:
#     angular_velocity (numpy.ndarray): Angular velocity in degrees per second.
#     time (numpy.ndarray): Time vector in seconds.
#     threshold (float): Minimum change in slope to be considered the onset of the turn.

#     Returns:
#     int: Index of the turn onset.
#     """
#     # Calculate the first derivative of angular velocity (slope)
#     velocity_slope = np.diff(angular_velocity) / np.diff(time[:len(angular_velocity)])
    
#     # Identify where the slope significantly changes (left tail)
#     # Look for the first point where the slope exceeds the threshold (positive or negative change)
#     onset_index = np.where(np.abs(velocity_slope) > threshold)[0][0]
    
#     # Return the time of the onset
#     onset_time = time[onset_index + 1]  # +1 because np.diff reduces length by 1
#     return onset_index, onset_time



# # Example usage with feet angular velocity
# onset_index, onset_time_v1 = detect_turn_onset(feet_ang_vel, time)

# print(f"Turn onset detected at index {onset_index} (time: {onset_time_v1:.2f} seconds)")


In [ ]:
# Plot the angular velocity of the feet
plt.figure(figsize=(5, 3))
plt.plot(time[:-1], feet_ang_vel, label="Feet Angular Velocity", color='b')
plt.axvline(x=onset_time, color='r', linestyle='--', label=f"Turn Onset at {onset_time:.2f}s")
plt.axvline(x=onset_time_v1, color='g', linestyle='--', label=f"Turn Onset at {onset_time_v1:.2f}s")
plt.xlabel('Time (seconds)')
plt.ylabel('Angular Velocity (deg/s)')
plt.title('Angular Velocity of Feet with Turn Onset')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

##### Export pdf with yaw angles and angular velocity plots

In [ ]:
# with PdfPages('turning_subjectwise_v2.pdf') as pdf:
#     full_data = [fma_control,fma_pwp]
#     data_label = ['Control','PwP']

#     for i,group in enumerate(full_data):
#         type_subject = data_label[i]
#         for file in group:

#            # Extract subject ID
#             if type_subject == 'Control':
#                 subject_id = re.search(r'_C(\d+)\.tsv$', file).group(1)
#                 subject_id = 'C'+subject_id
#                 color = 'blue'
#             else: 
#                 subject_id = re.search(r'_P(\d+)\.tsv$', file).group(1)
#                 subject_id = 'P'+subject_id


#             # Extract trial ID
#             trial = re.search(r'_FMA0(\d+)_', file).group(1)

#             flag,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE = read_file(file)
#             head_yaw,shoulders_yaw,pelvis_yaw,feet_yaw = yaw_angle_per_segment(LFHD, RFHD,LSHO, RSHO,LASI, RASI,LANK,RANK)

#                 # Prepare the figure
# #             plt.close('all')
# #             plt.ioff()
#             figprops = dict(dpi=100)

#             fig = plt.figure(**figprops)

#             ax = plt.axes([0.1, 0.65,0.65, 0.3])
#             ax.plot(time,head_yaw,label='head')
#             ax.plot(time,shoulders_yaw,label='shoulders')
#             ax.plot(time,pelvis_yaw,label='pelvis')
#             ax.plot(time,feet_yaw,label='feet')
#             ax.legend(loc = 'upper right')
#             ax.set_xlabel('Time')
#             ax.set_ylabel('Turn (yaw) deg')
#             ax.set_title('{} {} FMA {}'.format(type_subject,subject_id,trial))
#             ax.set_ylim(-200,200)

#             bx = plt.axes([0.1, 0.10, 0.65, 0.4], sharex=ax)
#             bx.plot(time[:-1],np.diff(head_yaw),label='head')
#             bx.plot(time[:-1],np.diff(shoulders_yaw),label='shoulders')
#             bx.plot(time[:-1],np.diff(pelvis_yaw),label='pelvis')
#             bx.plot(time[:-1],np.diff(feet_yaw),label='feet')
#             bx.legend(loc='upper right')
#             bx.set_xlabel('Time')
#             bx.set_ylabel('Angular velocity deg/s')
#             bx.set_ylim(-20,20)

#             pdf.savefig(fig)

In [ ]:
# # Plot angular velocities for head, shoulders, pelvis, and feet in subplots
# fig, axs = plt.subplots(4, 1, figsize=(8, 10), sharex=True)

# # Plot Head Angular Velocity
# axs[0].plot(time[:-1], head_ang_vel, label="Head Angular Velocity", color='g')
# axs[0].axvline(x=head_onset, color='r', linestyle='--', label=f"Turn Onset at {head_onset:.2f}s")
# axs[0].set_ylabel('Angular Velocity (deg/s)')
# axs[0].set_title('Head Angular Velocity with Turn Onset')
# axs[0].legend(loc='upper right')
# axs[0].grid(True)

# # Plot Shoulders Angular Velocity
# axs[1].plot(time[:-1], shoulders_ang_vel, label="Shoulders Angular Velocity", color='b')
# axs[1].axvline(x=shoulders_onset, color='r', linestyle='--', label=f"Turn Onset at {shoulders_onset:.2f}s")
# axs[1].set_ylabel('Angular Velocity (deg/s)')
# axs[1].set_title('Shoulders Angular Velocity with Turn Onset')
# axs[1].legend(loc='upper right')
# axs[1].grid(True)

# # Plot Pelvis Angular Velocity
# axs[2].plot(time[:-1], pelvis_ang_vel, label="Pelvis Angular Velocity", color='m')
# axs[2].axvline(x=pelvis_onset, color='r', linestyle='--', label=f"Turn Onset at {pelvis_onset:.2f}s")
# axs[2].set_ylabel('Angular Velocity (deg/s)')
# axs[2].set_title('Pelvis Angular Velocity with Turn Onset')
# axs[2].legend(loc='upper right')
# axs[2].grid(True)

# # Plot Feet Angular Velocity
# axs[3].plot(time[:-1], feet_ang_vel, label="Feet Angular Velocity", color='c')
# axs[3].axvline(x=onset_time, color='r', linestyle='--', label=f"Turn Onset at {onset_time:.2f}s")
# axs[3].set_xlabel('Time (seconds)')
# axs[3].set_ylabel('Angular Velocity (deg/s)')
# axs[3].set_title('Feet Angular Velocity with Turn Onset')
# axs[3].legend(loc='upper right')
# axs[3].grid(True)

# # Display the plots
# plt.tight_layout()
# plt.show()